# MOUNT DRIVE !!!

In [1]:
# Hpyerparameters
device = "cuda"
split = "train"
mbs = 1024

In [2]:
!pip install datasets
!pip install ctranslate2
!pip install OpenNMT-py=2.*
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 808.4 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.7/36.7 MB 29.6 MB/s eta 0:00:00
ERROR: Invalid requirement: 'OpenNMT-py=2.*'
Hint: = is not a valid operator. Did you mean == ?


In [3]:
import ctranslate2
import sentencepiece as spm
from datasets import load_dataset
from tqdm.auto import tqdm
import os
import torch

In [5]:
# get data
os.makedirs("/content/drive/My Drive/TinyStoriesTranslate/", exist_ok=True)
dataset = load_dataset("roneneldan/TinyStories", split=split)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

In [4]:
# get model
!wget https://object.pouta.csc.fi/OPUS-MT-models/en-de/opus-2020-02-26.zip
!unzip opus-2020-02-26.zip
!ct2-opus-mt-converter --model_dir . --output_dir ende_ctranslate2

--2024-03-15 18:05:31--  https://object.pouta.csc.fi/OPUS-MT-models/en-de/opus-2020-02-26.zip
Resolving object.pouta.csc.fi (object.pouta.csc.fi)... 86.50.254.18, 86.50.254.19
Connecting to object.pouta.csc.fi (object.pouta.csc.fi)|86.50.254.18|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 275458389 (263M) [application/zip]
Saving to: ‘opus-2020-02-26.zip’

opus-2020-02-26.zip 100%[===================>] 262.70M  20.2MB/s    in 17s     

2024-03-15 18:05:49 (15.4 MB/s) - ‘opus-2020-02-26.zip’ saved [275458389/275458389]

Archive:  opus-2020-02-26.zip
  inflating: README.md               
  inflating: LICENSE                 
  inflating: opus.spm32k-spm32k.transformer-align.model1.npz.best-perplexity.npz  
  inflating: opus.spm32k-spm32k.vocab.yml  
  inflating: opus.spm32k-spm32k.transformer-align.valid1.log  
  inflating: opus.spm32k-spm32k.transformer-align.train1.log  
  inflating: source.spm              
 extracting: source.tcmodel          
  inflating

In [8]:
sp = spm.SentencePieceProcessor()
sp.load("source.spm")

translator = ctranslate2.Translator(
    "ende_ctranslate2",
    compute_type="auto",
    device=device,
)
print(f"Using compute type: {translator.compute_type}")


Using compute type: int8_float16


In [9]:
def translate(start_ind, end_ind, verbose=True, mbs=128, split="train"):
    if verbose: print("Encoding...")
    encodings = []
    for story in dataset[start_ind:end_ind]["text"]:
      encodings.append(sp.encode(story, out_type=str))
    if verbose: print("Translating...")
    translations_encoded = translator.translate_batch(encodings, beam_size=1, max_batch_size=mbs)
    if verbose: print("Decoding...")
    translations = [sp.decode(tra.hypotheses[0]) for tra in translations_encoded]
    if verbose: print("Saving...")
    with open(f"/content/drive/My Drive/TinyStoriesTranslate/TS_{split}_de_{start_ind}_{end_ind}.csv", "w") as f:
        for story in translations:
            story = story.replace("▁", " ")
            if story[0] == " ":
              story = story[1:]
            f.write(story + "\n")
    if verbose: print("Done")

In [15]:
translate(0, 10000, mbs=256, split=split)

Encoding...
Translating...
Decoding...
Saving...
Done
